# Reinhold & Hekker 2020 Table 2

June 15, 2022  
Gully & Ryan H.

The goal of this notebook is to read in Table 2 of the K2 rotation period paper.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


sns.set_context('notebook', font_scale=1.5)
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

### Programmatically fetch the table from CDS

We want Table 2 from [this paper](https://ui.adsabs.harvard.edu/abs/2020A%26A...635A..43R/abstract).

In [ ]:
#! mkdir ../../data/Reinhold_Hekker2020
#! mv /home/gully/Downloads/table2.dat.gz ../../data/Reinhold_Hekker2020/
# Extract the table...
#! du -hs ../../data/Reinhold_Hekker2020/
#! head ../../data/Reinhold_Hekker2020/table2.dat

| Column Name | Unit | Description|
| -- | -- | -- | 
|EPIC| | |
|Campaign| |
|Teff| K| Effective Temperature|
|log g| dex| Surface Gravity|
|Prot| days  | Rotation period|
|ΔProt| days| Uncertainty in rotation period|
|hpeak| | ?|
|Rvar| %| Proportional to Amplitude |
|Kp|mag| Kepler magnitude|
|MG|mag| Absolute Magnitude |

In [ ]:
names = ['EPIC','Campaign','Teff','log g','Prot','ΔProt','hpeak','Rvar','Kp','MG']

In [ ]:
df = pd.read_csv('../../data/Reinhold_Hekker2020/table2.dat', 
                 delim_whitespace=True, names=names, na_values='---')

Looks good!  We see the same trend we had in our proposal figure 2.

## Select a subsample of sources

First search for some high amplitude variable stars

In [ ]:
criterion1 = (df.Prot < 7) & (df.Prot > 4)

In [ ]:
criterion1.sum()

In [ ]:
criterion2 = (df.Rvar > 1) & (df.Rvar < 6)

In [ ]:
criterion2.sum()

In [ ]:
criterion3 = (df.Teff > 4000) & (df.Teff < 4500)

In [ ]:
criterion3.sum()

In [ ]:
criteria = criterion1 & criterion2 & criterion3

In [ ]:
criteria.sum()

In [ ]:
#plt.plot(df.Prot, df.Rvar, '.', alpha=0.02);
plt.plot(df.Prot[criterion3], df.Rvar[criterion3], '.', alpha=0.1);
plt.plot(df.Prot[criteria], df.Rvar[criteria], '.', alpha=0.5);
#plt.ylim(3e2, 2e5)
plt.xlim(1e0, 1e2)
plt.xscale('log')
plt.yscale('log')
plt.xlabel('$P_{\mathrm{rot}}$')
plt.ylabel('$\propto$ Amplitude (%)')
plt.title('Reinhold & Hekker 2020 Table 2')

In [ ]:
df[criteria].head()

## Spot check the subsample

In [ ]:
import lightkurve as lk

In [ ]:
sr = lk.search_lightcurve("EPIC 201377225", author="EVEREST", mission="K2")

In [ ]:
assert len(sr) == 1

In [ ]:
lc=sr.download()

In [ ]:
lc = lc.remove_outliers(sigma=4,sigma_upper=3).normalize()

In [ ]:
pg = lc.to_periodogram(nterms=5)

In [ ]:
ax = pg.plot(view='period', scale='log')
ax.axvline(pg.period_at_max_power.value, linestyle='dotted', label=f'{pg.period_at_max_power:0.5f}')
ax.axvline(6.70, linestyle='dashed', label='6.7 d (Reinhold & Hekker 2020)', color = 'red')
ax.legend(fontsize = 12)

In [ ]:
pg.period_at_max_power

In [ ]:
ax = lc.plot()
pg.model(lc.time).plot(ax=ax)
pg.model(lc.time, frequency=pg.frequency_at_max_power/2).plot(ax=ax)